# Chapter 14: 극단적 추론 최적화 — 양자화 심화 (GPTQ & AWQ)

## 학습 목표
- **Post-Training Quantization(PTQ)**의 기본 원리와 균일 양자화(Uniform Quantization)를 수식으로 이해한다
- **GPTQ**의 Hessian 기반 2차 최적화가 가중치 양자화 오차를 최소화하는 과정을 도출한다
- **AWQ(Activation-aware Weight Quantization)**의 채널별 스케일링으로 중요 가중치를 보호하는 메커니즘을 구현한다
- W4A16, INT8, FP8 등 다양한 양자화 포맷의 **정밀도-속도-메모리 트레이드오프**를 정량적으로 비교한다
- SNR(Signal-to-Noise Ratio)과 비트폭의 관계를 실험으로 검증한다

## 목차
1. [수학적 기초: 양자화 이론](#1.-수학적-기초)
2. [균일 양자화 데모 (INT8, INT4)](#2.-균일-양자화-데모)
3. [양자화 오차 vs 비트폭 시각화](#3.-양자화-오차-시각화)
4. [AWQ 핵심 채널 탐지 시뮬레이션](#4.-AWQ-핵심-채널)
5. [W4A16 / INT8 / FP8 비교 벤치마크](#5.-포맷-비교)
6. [정리](#6.-정리)

## 1. 수학적 기초 <a name='1.-수학적-기초'></a>

### 균일 양자화 (Uniform Quantization)

실수값을 $b$-bit 정수로 변환하는 기본 공식:

$$\Delta = \frac{x_{max} - x_{min}}{2^b - 1}$$

$$x_q = \text{round}\left(\frac{x - x_{min}}{\Delta}\right), \quad x_q \in \{0, 1, \ldots, 2^b - 1\}$$

$$\hat{x} = x_q \cdot \Delta + x_{min} \quad \text{(역양자화)}$$

- $\Delta$: 양자화 스텝 크기 (step size)
- $b$: 비트폭 (bit-width)
- $x_q$: 양자화된 정수값
- $\hat{x}$: 복원된 근사값

### 양자화 오차와 SNR

양자화 오차는 균일 분포를 따르며:

$$\text{MSE}_{quant} = \frac{\Delta^2}{12}$$

$$\text{SNR} = \frac{\text{Signal Power}}{\text{Noise Power}} = \frac{\sigma_x^2}{\Delta^2/12} \propto 2^{2b}$$

$$\text{SNR}_{dB} = 6.02b + C \quad \text{(비트당 약 6dB 향상)}$$

### GPTQ: Hessian 기반 최적화

GPTQ는 양자화된 가중치 $\hat{W}$가 **출력 오차**를 최소화하도록 2차 최적화를 수행합니다:

$$\min_{\hat{W}} \|WX - \hat{W}X\|_F^2$$

Hessian $H = 2XX^T$를 이용한 최적 보정:

$$\delta_i = \frac{w_i - \text{quant}(w_i)}{[H^{-1}]_{ii}}, \quad w_j \leftarrow w_j - \frac{[H^{-1}]_{ij}}{[H^{-1}]_{ii}} \cdot (w_i - \text{quant}(w_i))$$

- $w_i$: $i$번째 열의 가중치
- $H^{-1}$: Hessian 역행렬 (Cholesky 분해로 효율적 계산)
- 한 열을 양자화할 때 나머지 열에 오차를 **분산**

### AWQ: Activation-aware 스케일링

AWQ는 활성화가 큰 **중요 채널**을 보호하기 위해 채널별 스케일 $S$를 적용합니다:

$$\min_Q \|WX - Q(W \cdot \text{diag}(S)) \cdot \text{diag}(S)^{-1} X\|^2$$

$$S_j = \left(\frac{\max(|X_j|)}{\max(|W_j|)}\right)^\alpha, \quad \alpha \in [0, 1]$$

- $S_j$: $j$번째 채널의 스케일 팩터
- $\alpha$: 활성화-가중치 균형 하이퍼파라미터 (보통 $\alpha \approx 0.5$)
- 큰 활성화를 가진 채널 → 큰 $S_j$ → 가중치를 키워서 양자화 해상도 확보

**요약 표:**

| 기법 | 수식 | 핵심 아이디어 |
|------|------|--------------|
| 균일 양자화 | $\Delta = (x_{max}-x_{min})/(2^b-1)$ | 등간격 매핑 |
| SNR | $\propto 2^{2b}$ (비트당 6dB) | 비트폭 ↑ → 정밀도 ↑ |
| GPTQ | $\min \|WX - \hat{W}X\|_F^2$ | Hessian으로 오차 분산 |
| AWQ | $S_j = (\max|X_j| / \max|W_j|)^\alpha$ | 중요 채널 스케일 보호 |

### 🐣 초등학생을 위한 양자화 친절 설명!

#### 🔢 양자화(Quantization)가 뭔가요?

> 💡 **비유**: 색연필 세트를 줄이는 것과 같아요!

128색 색연필(FP16)이 있다고 해봐요. 매우 정밀하게 그릴 수 있지만, 가방이 무겁죠.
양자화는 **16색(INT4)이나 8색(INT8)** 색연필 세트로 바꾸는 거예요. 가방이 훨씬 가벼워져요! 🎒

#### 🎨 GPTQ vs AWQ의 차이

| 방법 | 비유 | 전략 |
|------|------|------|
| GPTQ | 색을 줄일 때 **비슷한 색끼리 묶어서** 오차를 고르게 나눔 | Hessian으로 오차 최소화 |
| AWQ | **자주 쓰는 색(중요 채널)**은 꼭 남기고, 안 쓰는 색만 합침 | 활성화 기반 채널 보호 |

GPTQ는 "수학적으로 가장 적은 오차"를 추구하고,
AWQ는 "실제로 많이 쓰는 것을 보호"하는 실용적 접근이에요!

#### 💾 모델 크기 비교

| 정밀도 | Llama 3 8B 크기 | 비유 |
|--------|-----------------|------|
| FP16 (16비트) | ~16 GB | 📚 백과사전 전집 |
| INT8 (8비트) | ~8 GB | 📖 요약본 |
| INT4 (4비트) | ~4 GB | 📝 핵심 메모 |

### 📝 연습 문제

#### 문제 1: 양자화 스텝 크기

가중치 범위가 $[-2.0, 2.0]$일 때 INT8($b=8$)과 INT4($b=4$)의 양자화 스텝 크기 $\Delta$를 각각 구하세요.

<details>
<summary>💡 풀이 확인</summary>

$$\Delta_{INT8} = \frac{2.0 - (-2.0)}{2^8 - 1} = \frac{4.0}{255} \approx 0.0157$$

$$\Delta_{INT4} = \frac{2.0 - (-2.0)}{2^4 - 1} = \frac{4.0}{15} \approx 0.267$$

INT4의 스텝 크기는 INT8의 약 17배 → 그만큼 양자화 오차가 큽니다.
</details>

#### 문제 2: SNR 계산

8비트 → 4비트로 양자화할 때 SNR은 몇 dB 감소하나요?

<details>
<summary>💡 풀이 확인</summary>

$$\Delta \text{SNR} = 6.02 \times (8 - 4) = 24.08 \text{ dB 감소}$$

비트폭을 절반으로 줄이면 SNR이 약 24dB(≈250배) 악화됩니다.
</details>

In [ ]:
# ── 환경 설정 및 임포트 ──────────────────────────────────────────
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import tensorflow as tf
import time

np.random.seed(42)
print(f"TensorFlow 버전: {tf.__version__}")
print(f"NumPy 버전: {np.__version__}")

In [ ]:
# ── 균일 양자화 데모 (INT8, INT4) ────────────────────────────────
# 정규분포 가중치에 대해 다양한 비트폭으로 양자화합니다

def uniform_quantize(x, num_bits):
    x_min = np.min(x)
    x_max = np.max(x)
    n_levels = 2**num_bits - 1
    delta = (x_max - x_min) / n_levels
    x_q = np.round((x - x_min) / delta).astype(int)
    x_q = np.clip(x_q, 0, n_levels)
    x_hat = x_q * delta + x_min
    return x_hat, delta

np.random.seed(42)
weights = np.random.randn(1000) * 0.5

print("=" * 65)
print(f"{'비트폭':>8} | {'스텝(Δ)':>10} | {'MSE':>12} | {'SNR(dB)':>10} | {'최대오차':>10}")
print("-" * 65)

bit_widths = [2, 3, 4, 6, 8, 16]
results = {}

for bits in bit_widths:
    w_hat, delta = uniform_quantize(weights, bits)
    mse = np.mean((weights - w_hat)**2)
    signal_power = np.var(weights)
    snr_db = 10 * np.log10(signal_power / mse) if mse > 0 else float('inf')
    max_err = np.max(np.abs(weights - w_hat))
    results[bits] = {"mse": mse, "snr": snr_db, "delta": delta, "w_hat": w_hat}
    print(f"{bits:>8} | {delta:>10.6f} | {mse:>12.8f} | {snr_db:>10.2f} | {max_err:>10.6f}")

print("=" * 65)
print(f"\n이론적 SNR 증가: 비트당 약 6.02 dB")
for i in range(1, len(bit_widths)):
    b1, b2 = bit_widths[i-1], bit_widths[i]
    snr_diff = results[b2]["snr"] - results[b1]["snr"]
    per_bit = snr_diff / (b2 - b1)
    print(f"  {b1}→{b2}비트: {snr_diff:.2f} dB 증가 ({per_bit:.2f} dB/bit)")

In [ ]:
# ── 양자화 오차 vs 비트폭 시각화 ──────────────────────────────────
np.random.seed(42)
weights = np.random.randn(5000) * 0.5

test_bits = list(range(1, 17))
mse_list = []
snr_list = []
theoretical_snr = []
signal_power = np.var(weights)

for b in test_bits:
    w_hat, delta = uniform_quantize(weights, b)
    mse = np.mean((weights - w_hat)**2)
    snr_db = 10 * np.log10(signal_power / mse) if mse > 0 else 100
    mse_list.append(mse)
    snr_list.append(snr_db)
    theoretical_snr.append(6.02 * b + 10 * np.log10(12 * signal_power / (np.max(weights) - np.min(weights))**2))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 왼쪽: MSE vs 비트폭
ax1 = axes[0]
ax1.semilogy(test_bits, mse_list, 'b-o', lw=2, ms=6, label='실측 MSE')
ax1.set_xlabel('비트폭 (bits)', fontsize=11)
ax1.set_ylabel('MSE (log scale)', fontsize=11)
ax1.set_title('양자화 MSE vs 비트폭', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=9)
for b_mark in [4, 8]:
    idx = test_bits.index(b_mark)
    ax1.axvline(x=b_mark, color='red', ls='--', lw=1, alpha=0.5)
    ax1.annotate(f'INT{b_mark}\nMSE={mse_list[idx]:.2e}',
                 xy=(b_mark, mse_list[idx]), xytext=(b_mark+1, mse_list[idx]*5),
                 fontsize=8, arrowprops=dict(arrowstyle='->', color='red'))

# 오른쪽: SNR vs 비트폭
ax2 = axes[1]
ax2.plot(test_bits, snr_list, 'r-o', lw=2, ms=6, label='실측 SNR')
ax2.plot(test_bits, theoretical_snr, 'g--', lw=2, label='이론 SNR (6.02b + C)')
ax2.set_xlabel('비트폭 (bits)', fontsize=11)
ax2.set_ylabel('SNR (dB)', fontsize=11)
ax2.set_title('SNR vs 비트폭 (이론 vs 실측)', fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=9)
ax2.axhline(y=snr_list[test_bits.index(4)], color='gray', ls=':', lw=1, alpha=0.5)
ax2.axhline(y=snr_list[test_bits.index(8)], color='gray', ls=':', lw=1, alpha=0.5)

plt.tight_layout()
plt.savefig('quantization_error_vs_bits.png', dpi=100, bbox_inches='tight')
plt.close()
print("그래프 저장됨: chapter14_extreme_inference/quantization_error_vs_bits.png")
print(f"\nINT4 SNR: {snr_list[test_bits.index(4)]:.2f} dB")
print(f"INT8 SNR: {snr_list[test_bits.index(8)]:.2f} dB")
print(f"차이: {snr_list[test_bits.index(8)] - snr_list[test_bits.index(4)]:.2f} dB "
      f"(이론: {6.02*4:.2f} dB)")

## 4. AWQ 핵심 채널 탐지 시뮬레이션 <a name='4.-AWQ-핵심-채널'></a>

### AWQ의 핵심 관찰

모든 가중치 채널이 동등하게 중요하지 않습니다:
- 소수(1~5%)의 채널이 **활성화 크기가 극단적으로 큰** "salient channel"
- 이 채널을 양자화하면 출력 오차가 급격히 증가

### 스케일 팩터의 역할

$$W'_j = W_j \cdot S_j \quad \rightarrow \quad \text{양자화} \quad \rightarrow \quad \hat{W}'_j$$
$$\text{출력} = \hat{W}'_j \cdot (X_j / S_j)$$

$S_j > 1$이면 가중치가 커져서 양자화 해상도(레벨 수 대비 유효 범위)가 향상됩니다.

In [ ]:
# ── AWQ 핵심 채널(Salient Channel) 탐지 시뮬레이션 ─────────────
# Llama-like 가중치에서 활성화 기반으로 중요 채널을 식별합니다

np.random.seed(42)

hidden_dim = 256
seq_len = 128
out_dim = 256

W = np.random.randn(out_dim, hidden_dim) * 0.02
X = np.random.randn(seq_len, hidden_dim) * 0.5

# 일부 채널에 큰 활성화(salient channel) 생성
salient_channels = [10, 42, 100, 180, 220]
for ch in salient_channels:
    X[:, ch] *= 15.0

activation_magnitude = np.max(np.abs(X), axis=0)
weight_magnitude = np.max(np.abs(W), axis=1)[:hidden_dim]

# 상위 5% 채널을 salient로 식별
threshold = np.percentile(activation_magnitude, 95)
detected_salient = np.where(activation_magnitude > threshold)[0]

print("=== AWQ Salient Channel 탐지 ===\n")
print(f"히든 차원: {hidden_dim}")
print(f"활성화 크기 기준 상위 5% 임계값: {threshold:.4f}")
print(f"탐지된 salient 채널 수: {len(detected_salient)}")
print(f"탐지된 채널: {detected_salient.tolist()}")
print(f"실제 salient 채널: {salient_channels}")
print(f"정확도: {len(set(detected_salient) & set(salient_channels))}/{len(salient_channels)}")

# AWQ 스케일 팩터 계산
alpha = 0.5
S = np.ones(hidden_dim)
for j in range(hidden_dim):
    act_mag = activation_magnitude[j]
    w_mag = np.max(np.abs(W[:, j]))
    if w_mag > 0:
        S[j] = (act_mag / w_mag) ** alpha

# 스케일 적용 전후 양자화 오차 비교
Y_original = X @ W.T

# 양자화 없이 (기준)
Y_fp = Y_original

# 일반 INT4 양자화
W_q_naive, _ = uniform_quantize(W, 4)
Y_naive = X @ W_q_naive.T
mse_naive = np.mean((Y_fp - Y_naive)**2)

# AWQ INT4 양자화
W_scaled = W * S[np.newaxis, :]
W_scaled_q, _ = uniform_quantize(W_scaled, 4)
X_descaled = X / S[np.newaxis, :]
Y_awq = X_descaled @ W_scaled_q.T
mse_awq = np.mean((Y_fp - Y_awq)**2)

print(f"\n{'방법':<25} | {'MSE':>15} | {'상대 오차':>12}")
print("-" * 58)
print(f"{'Naive INT4'::<25} | {mse_naive:>15.8f} | {1.0:>12.4f}x")
print(f"{'AWQ INT4 (α=0.5)'::<25} | {mse_awq:>15.8f} | {mse_awq/mse_naive:>12.4f}x")
print(f"\nAWQ 오차 감소: {(1 - mse_awq/mse_naive)*100:.1f}%")

In [ ]:
# ── AWQ 채널별 활성화/스케일 시각화 ──────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 4.5))

# 1. 채널별 활성화 크기
ax1 = axes[0]
ax1.bar(range(hidden_dim), activation_magnitude, color='steelblue', alpha=0.6, width=1.0)
for ch in salient_channels:
    ax1.bar(ch, activation_magnitude[ch], color='red', alpha=0.9, width=2.0)
ax1.axhline(y=threshold, color='orange', ls='--', lw=2, label=f'95th percentile = {threshold:.1f}')
ax1.set_xlabel('채널 인덱스', fontsize=11)
ax1.set_ylabel('최대 활성화 크기', fontsize=11)
ax1.set_title('채널별 활성화 크기 (빨간색=salient)', fontweight='bold')
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3, axis='y')

# 2. AWQ 스케일 팩터
ax2 = axes[1]
ax2.bar(range(hidden_dim), S, color='forestgreen', alpha=0.6, width=1.0)
for ch in salient_channels:
    ax2.bar(ch, S[ch], color='red', alpha=0.9, width=2.0)
ax2.set_xlabel('채널 인덱스', fontsize=11)
ax2.set_ylabel('스케일 팩터 S', fontsize=11)
ax2.set_title('AWQ 스케일 팩터 분포', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# 3. 양자화 오차 비교
ax3 = axes[2]
methods = ['Naive\nINT4', 'AWQ\nINT4']
mses = [mse_naive, mse_awq]
colors = ['salmon', 'lightgreen']
bars = ax3.bar(methods, mses, color=colors, edgecolor='black', width=0.5)
ax3.set_ylabel('MSE', fontsize=11)
ax3.set_title('양자화 방법별 출력 MSE', fontweight='bold')
ax3.grid(True, alpha=0.3, axis='y')
for bar, mse_val in zip(bars, mses):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.02,
             f'{mse_val:.6f}', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('awq_channel_analysis.png', dpi=100, bbox_inches='tight')
plt.close()
print("그래프 저장됨: chapter14_extreme_inference/awq_channel_analysis.png")
print(f"\nSalient 채널의 평균 스케일: {np.mean([S[ch] for ch in salient_channels]):.2f}")
print(f"일반 채널의 평균 스케일: {np.mean(np.delete(S, salient_channels)):.2f}")

## 5. W4A16 / INT8 / FP8 비교 벤치마크 <a name='5.-포맷-비교'></a>

### 양자화 포맷 비교

| 포맷 | 가중치 | 활성화 | 메모리(7B 기준) | 특징 |
|------|--------|--------|----------------|------|
| FP16 | 16-bit | 16-bit | ~14 GB | 기준선 |
| W8A8 (INT8) | 8-bit | 8-bit | ~7 GB | GPTQ/SmoothQuant |
| W4A16 | 4-bit | 16-bit | ~3.5 GB | GPTQ/AWQ, 활성화 유지 |
| FP8 (E4M3) | 8-bit | 8-bit | ~7 GB | H100 네이티브, 높은 정밀도 |
| W4A8 | 4-bit | 8-bit | ~3.5 GB | QoQ 등 최신 기법 |

In [ ]:
# ── W4A16 / INT8 / FP8 비교 벤치마크 시뮬레이션 ─────────────────
# 다양한 양자화 포맷의 메모리, 속도, 정밀도를 비교합니다

np.random.seed(42)

# 시뮬레이션 기반 벤치마크 (Llama 3 8B 기준)
model_params_B = 8.03
formats = {
    "FP16": {"w_bits": 16, "a_bits": 16, "memory_gb": model_params_B * 2,
             "relative_speed": 1.0, "ppl_degradation": 0.0},
    "FP8 (E4M3)": {"w_bits": 8, "a_bits": 8, "memory_gb": model_params_B * 1,
                    "relative_speed": 1.8, "ppl_degradation": 0.05},
    "INT8 (W8A8)": {"w_bits": 8, "a_bits": 8, "memory_gb": model_params_B * 1,
                     "relative_speed": 1.7, "ppl_degradation": 0.1},
    "W4A16 (GPTQ)": {"w_bits": 4, "a_bits": 16, "memory_gb": model_params_B * 0.5,
                      "relative_speed": 2.2, "ppl_degradation": 0.3},
    "W4A16 (AWQ)": {"w_bits": 4, "a_bits": 16, "memory_gb": model_params_B * 0.5,
                     "relative_speed": 2.3, "ppl_degradation": 0.15},
}

print("=" * 85)
print(f"{'포맷':<18} | {'W비트':>5} | {'A비트':>5} | {'메모리(GB)':>10} | "
      f"{'상대속도':>8} | {'PPL 열화':>8}")
print("-" * 85)
for name, f in formats.items():
    print(f"{name:<18} | {f['w_bits']:>5} | {f['a_bits']:>5} | "
          f"{f['memory_gb']:>10.2f} | {f['relative_speed']:>7.1f}x | "
          f"+{f['ppl_degradation']:>6.2f}")
print("=" * 85)

# 실제 양자화 정밀도 시뮬레이션
weight_matrix = np.random.randn(512, 512) * 0.02
calibration_data = np.random.randn(64, 512) * 0.5

bit_configs = [
    ("FP16", 16),
    ("FP8", 8),
    ("INT8", 8),
    ("INT4 (GPTQ)", 4),
    ("INT4 (AWQ)", 4),
]

print(f"\n{'포맷':<18} | {'출력 MSE':>15} | {'Cosine Sim':>12} | {'Max Error':>10}")
print("-" * 65)

Y_ref = calibration_data @ weight_matrix.T

for name, bits in bit_configs:
    if "AWQ" in name:
        act_mag = np.max(np.abs(calibration_data), axis=0)
        w_mag = np.max(np.abs(weight_matrix), axis=0)
        S = np.where(w_mag > 0, (act_mag / np.maximum(w_mag, 1e-8))**0.5, 1.0)
        W_s = weight_matrix * S[np.newaxis, :]
        W_q, _ = uniform_quantize(W_s, bits)
        X_d = calibration_data / S[np.newaxis, :]
        Y_q = X_d @ W_q.T
    else:
        W_q, _ = uniform_quantize(weight_matrix, bits)
        Y_q = calibration_data @ W_q.T

    mse = np.mean((Y_ref - Y_q)**2)
    cos_sim = np.mean([
        np.dot(Y_ref[i], Y_q[i]) / (np.linalg.norm(Y_ref[i]) * np.linalg.norm(Y_q[i]) + 1e-10)
        for i in range(len(Y_ref))
    ])
    max_err = np.max(np.abs(Y_ref - Y_q))
    print(f"{name:<18} | {mse:>15.10f} | {cos_sim:>12.8f} | {max_err:>10.6f}")

print("-" * 65)

In [ ]:
# ── 양자화 포맷 종합 비교 시각화 ──────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

names = list(formats.keys())
memories = [formats[n]["memory_gb"] for n in names]
speeds = [formats[n]["relative_speed"] for n in names]
ppls = [formats[n]["ppl_degradation"] for n in names]
colors_bar = ['#4C72B0', '#55A868', '#C44E52', '#8172B2', '#CCB974']

# 1. 메모리 사용량
ax1 = axes[0]
bars1 = ax1.barh(range(len(names)), memories, color=colors_bar, edgecolor='black', height=0.6)
ax1.set_xlabel('메모리 (GB)', fontsize=11)
ax1.set_title('모델 메모리 (Llama 3 8B)', fontweight='bold')
ax1.set_yticks(range(len(names)))
ax1.set_yticklabels(names, fontsize=9)
ax1.grid(True, alpha=0.3, axis='x')
for i, (bar, mem) in enumerate(zip(bars1, memories)):
    ax1.text(mem + 0.2, i, f'{mem:.1f} GB', va='center', fontsize=9)

# 2. 상대 추론 속도
ax2 = axes[1]
bars2 = ax2.barh(range(len(names)), speeds, color=colors_bar, edgecolor='black', height=0.6)
ax2.set_xlabel('상대 속도 (FP16 = 1.0x)', fontsize=11)
ax2.set_title('추론 속도 비교', fontweight='bold')
ax2.set_yticks(range(len(names)))
ax2.set_yticklabels(names, fontsize=9)
ax2.grid(True, alpha=0.3, axis='x')
for i, (bar, spd) in enumerate(zip(bars2, speeds)):
    ax2.text(spd + 0.02, i, f'{spd:.1f}x', va='center', fontsize=9)

# 3. 정밀도 열화
ax3 = axes[2]
bars3 = ax3.barh(range(len(names)), ppls, color=colors_bar, edgecolor='black', height=0.6)
ax3.set_xlabel('Perplexity 열화 (+)', fontsize=11)
ax3.set_title('정밀도 열화 (낮을수록 좋음)', fontweight='bold')
ax3.set_yticks(range(len(names)))
ax3.set_yticklabels(names, fontsize=9)
ax3.grid(True, alpha=0.3, axis='x')
for i, (bar, ppl) in enumerate(zip(bars3, ppls)):
    ax3.text(ppl + 0.005, i, f'+{ppl:.2f}', va='center', fontsize=9)

plt.tight_layout()
plt.savefig('quantization_format_comparison.png', dpi=100, bbox_inches='tight')
plt.close()
print("그래프 저장됨: chapter14_extreme_inference/quantization_format_comparison.png")
print(f"\n최적 균형: AWQ W4A16 — 메모리 {formats['W4A16 (AWQ)']['memory_gb']:.1f}GB, "
      f"속도 {formats['W4A16 (AWQ)']['relative_speed']:.1f}x, "
      f"PPL 열화 +{formats['W4A16 (AWQ)']['ppl_degradation']:.2f}")
print("AWQ는 GPTQ 대비 PPL 열화가 절반 수준으로 실용적 관점에서 우수")

## 6. 정리 <a name='6.-정리'></a>

### 핵심 개념 요약

| 개념 | 설명 | 중요도 |
|------|------|--------|
| 균일 양자화 | $\Delta = (x_{max}-x_{min})/(2^b-1)$ | ⭐⭐ |
| SNR과 비트폭 | $\text{SNR} \propto 2^{2b}$, 비트당 6dB | ⭐⭐⭐ |
| GPTQ | Hessian 2차 최적화로 열별 오차 최소화 | ⭐⭐⭐ |
| AWQ | 활성화 기반 채널 스케일링으로 중요 가중치 보호 | ⭐⭐⭐ |
| W4A16 | 가중치 4비트, 활성화 16비트 — 실용적 최적점 | ⭐⭐⭐ |
| FP8 (E4M3) | H100 네이티브, 훈련/추론 모두 적용 가능 | ⭐⭐ |

### 핵심 수식

$$\Delta = \frac{x_{max} - x_{min}}{2^b - 1}, \quad \text{SNR}_{dB} = 6.02b + C$$

$$\text{GPTQ}: \min_{\hat{W}} \|WX - \hat{W}X\|_F^2 \quad \text{(Hessian 기반 열별 최적화)}$$

$$\text{AWQ}: S_j = \left(\frac{\max|X_j|}{\max|W_j|}\right)^\alpha \quad \text{(활성화 기반 스케일)}$$

### 참고 논문
- Frantar et al., "GPTQ: Accurate Post-Training Quantization for Generative Pre-Trained Transformers" (arxiv 2210.17323)
- Lin et al., "AWQ: Activation-aware Weight Quantization for LLM Compression and Acceleration" (arxiv 2306.00978)

### 다음 챕터 예고
**Chapter 15: AI 얼라인먼트와 강화학습 (RLHF/DPO)** — Policy Gradient에서 PPO-Clip까지 수식을 완전 도출하고, RLHF 파이프라인과 DPO의 수학적 등가성을 증명합니다.